In [10]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import os

In [3]:
randburg = pd.read_csv('Randburg Links - Private Properties.csv')

In [4]:
randburg

,Link
0,https://www.privateproperty.co.za/for-sale/gau...
1,https://www.privateproperty.co.za/for-sale/gau...
2,https://www.privateproperty.co.za/for-sale/gau...
3,https://www.privateproperty.co.za/for-sale/gau...
4,https://www.privateproperty.co.za/for-sale/gau...
...,...
1188,https://www.privateproperty.co.za/for-sale/gau...
1189,https://www.privateproperty.co.za/for-sale/gau...
1190,https://www.privateproperty.co.za/for-sale/gau...
1191,https://www.privateproperty.co.za/for-sale/gau...


In [5]:
date_time = datetime.datetime.now()
print(date_time)

2024-02-15 07:20:36.435881


In [6]:
# Function to extract data from a single link
def extract_data_from_link(link):
    # Setting up Chrome options
    chrome_options = Options()
    chrome_options.add_argument('--headless')  # Run in headless mode
    chrome_options.add_argument('--disable-gpu')  # Disable GPU acceleration

    # Initialize the WebDriver
    driver = webdriver.Chrome(options=chrome_options)
    
    driver.get(link)
    #time.sleep(5)  # Adding a delay to ensure the page loads properly

    # Create BeautifulSoup object
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Find all listing results
    listings = soup.find_all('a', class_='listingResult row')

    # Extracting data from listings
    results = []
    for listing in listings:
        title = listing['title']
        href = listing['href']

        # Store the extracted information as a dictionary
        result = {
            'Title': title,
            'Link': href
        }
        results.append(result)

    driver.quit()  # Quit the driver after use
    return results

# Function to loop through all links and extract data
def extract_data_from_all_links(randburg):
    all_results = []
    for link in randburg['Link']:
        results = extract_data_from_link(link)
        all_results.extend(results)
    return all_results

# Extracting data from all links
final_data = extract_data_from_all_links(randburg)

# Creating a new dataframe from the extracted data
final_df = pd.DataFrame(final_data)
final_df.head()

,Title,Link
0,2 Bed Flat in Cheltondale,/for-sale/gauteng/johannesburg/rosebank-and-pa...
1,5 Bed House in Craighall Park,/for-sale/gauteng/johannesburg/rosebank-and-pa...
2,4 Bed House in Parkview,/for-sale/gauteng/johannesburg/rosebank-and-pa...
3,14 Bed House in Parktown North,/for-sale/gauteng/johannesburg/rosebank-and-pa...
4,2 Bed Apartment in Gresswold,/for-sale/gauteng/johannesburg/rosebank-and-pa...


In [7]:
date_time = datetime.datetime.now()
print(date_time)

2024-02-15 10:48:02.263739


In [8]:
randburg_property_list = final_df
randburg_property_list['Href'] = 'https://www.privateproperty.co.za'+final_df['Link']
randburg_property_list = randburg_property_list.drop('Link', axis=1)
randburg_property_list.head()

,Title,Href
0,2 Bed Flat in Cheltondale,https://www.privateproperty.co.za/for-sale/gau...
1,5 Bed House in Craighall Park,https://www.privateproperty.co.za/for-sale/gau...
2,4 Bed House in Parkview,https://www.privateproperty.co.za/for-sale/gau...
3,14 Bed House in Parktown North,https://www.privateproperty.co.za/for-sale/gau...
4,2 Bed Apartment in Gresswold,https://www.privateproperty.co.za/for-sale/gau...


In [9]:
randburg_property_list.to_csv('property_links.csv')

In [10]:
a = randburg_property_list

In [11]:
date_time = datetime.datetime.now()
print(date_time)

2024-02-15 10:48:02.465265


In [12]:
# Configure Chrome options for headless mode
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--disable-gpu')  # Disable GPU acceleration

driver = webdriver.Chrome(options=chrome_options)

# Iterate over each URL in the 'Href' column
for url in a['Href']:
    # Load the URL
    driver.get(url)
    
    # Extract the HTML content of the page
    page_source = driver.page_source
    
    # Create BeautifulSoup object
    soup = BeautifulSoup(page_source, 'html.parser')
    
    # Find all span elements with class 'attributeLabel' for column names
    columns = soup.find_all('span', class_='attributeLabel')
    
    # Find all div elements with class 'propAttrValue' for values
    values = soup.find_all('div', class_='propAttrValue')
    
    # Find all span elements with class 'mainFeature' for main values
    main_values = soup.find_all('span', class_='mainFeature')
    
    # Find the span element with class 'detailsPrice' for the price
    price_element = soup.find('span', class_='detailsPrice')
    price = price_element.text.strip() if price_element else None
    
    # Create a dictionary to store the extracted data
    data_dict = {}
    
    # Extract data and store it in the dictionary
    for column, value in zip(columns, values):
        column_text = column.get_text(strip=True)
        value_text = value.get_text(strip=True)
        data_dict[column_text] = value_text
    
    # Extract main values and store them in the dictionary
    main_value_list = []  # List to store all main values
    for main_value in main_values:
        main_value_text = main_value.get_text(strip=True)
        main_value_list.append(main_value_text)  # Add main value to the list
    
    # Update the DataFrame with the extracted data
    for column, value in data_dict.items():
        # If the column does not exist, create it
        if column not in a.columns:
            a[column] = None
        # Set the value in the DataFrame
        a.loc[a['Href'] == url, column] = value
    
    # Add main values to the DataFrame
    for i, main_value in enumerate(main_value_list):
        main_column_name = f'Main Value {i+1}'  # Create column name for main value
        a.loc[a['Href'] == url, main_column_name] = main_value
    
    # Add price to the DataFrame
    a.loc[a['Href'] == url, 'Price'] = price
    
# Close the webdriver once done
driver.quit()


In [13]:
date_time = datetime.datetime.now()
print(date_time)

2024-02-15 23:15:20.596502


In [14]:
final_list = a.fillna("None")
final_list['Main Features'] = final_list['Main Value 1']+final_list['Main Value 2']
final_list.head()

,Title,Href,Bedrooms,Bathrooms,Covered Parkings,Lounges,Storeys,Access Gate,Balcony,BIC,...,Furnished,Tv,Tennis Court,Spa Bath,Squash Court,Sea View,Jetty Berth,Borehole,Electricity Included,Main Features
0,2 Bed Flat in Cheltondale,https://www.privateproperty.co.za/for-sale/gau...,2,1,1,1,1,✔,✔,✔,...,None,None,None,None,None,None,None,None,None,Floor Area75 m²RatesR 2000
1,5 Bed House in Craighall Park,https://www.privateproperty.co.za/for-sale/gau...,5,3.5,None,2,None,✔,None,None,...,None,None,None,None,None,None,None,None,None,Floor Area425 m²Land Area1983 m²
2,4 Bed House in Parkview,https://www.privateproperty.co.za/for-sale/gau...,4,3,None,2,None,✔,None,None,...,None,None,None,None,None,None,None,None,None,Land Area1020 m²RatesR 3739
3,14 Bed House in Parktown North,https://www.privateproperty.co.za/for-sale/gau...,14,13,None,5,None,None,None,None,...,None,None,None,None,None,None,None,None,None,Land Area2900 m²None
4,2 Bed Apartment in Gresswold,https://www.privateproperty.co.za/for-sale/gau...,2,2,2,None,None,✔,None,✔,...,None,None,None,None,None,None,None,None,None,Floor Area72 m²RatesR 386


In [15]:
# Function to extract values from text
def extract_info(text):
    # Initialize values
    floor_area = None
    land_area = None
    rates = None
    levy = None

    # Check for presence of keywords and extract the corresponding values
    if 'Floor Area' in text:
        floor_area = text.split('Floor Area')[1].split('Land Area')[0].strip()
    if 'Land Area' in text:
        land_area = text.split('Land Area')[1].split('Rates')[0].split('Levy')[0].strip()
    if 'Rates' in text:
        rates = text.split('Rates')[1].split('Levy')[0].strip()
    if 'Levy' in text:
        levy = text.split('Levy')[1].strip()

    return floor_area, land_area, rates, levy

# Apply the function to the 'Text' column and create new columns
final_list[['Floor Area', 'Land Area', 'Rates', 'Levy']] = final_list['Main Features'].apply(lambda x: pd.Series(extract_info(x)))

# Drop the original 'Text' column if not needed anymore
final_list.drop(['Main Features', 'Main Value 1', 'Main Value 2'], axis=1, inplace=True)

In [16]:
final_list = final_list.applymap(lambda x: '' if pd.isna(x) else x)

In [17]:
from urllib.parse import urlparse, parse_qs

# Function to extract information from the link
def extract_info(link):
    parsed_url = urlparse(link)
    query = parse_qs(parsed_url.query)
    path_components = parsed_url.path.split('/')

    province = path_components[2]
    city = path_components[3]
    suburb = path_components[4]
    area = path_components[5]

    return province, city, suburb, area

# Apply the function to the 'Link' column and create new columns
final_list[['Province', 'City', 'Suburb', 'Area']] = final_list['Href'].apply(lambda x: pd.Series(extract_info(x)))

In [18]:
final_list

,Title,Href,Bedrooms,Bathrooms,Covered Parkings,Lounges,Storeys,Access Gate,Balcony,BIC,...,Borehole,Electricity Included,Floor Area,Land Area,Rates,Levy,Province,City,Suburb,Area
0,2 Bed Flat in Cheltondale,https://www.privateproperty.co.za/for-sale/gau...,2,1,1,1,1,✔,✔,✔,...,None,None,75 m²RatesR 2000,,R 2000,,gauteng,johannesburg,rosebank-and-parktown,orchards
1,5 Bed House in Craighall Park,https://www.privateproperty.co.za/for-sale/gau...,5,3.5,None,2,None,✔,None,None,...,None,None,425 m²,1983 m²,,,gauteng,johannesburg,rosebank-and-parktown,craighall-park
2,4 Bed House in Parkview,https://www.privateproperty.co.za/for-sale/gau...,4,3,None,2,None,✔,None,None,...,None,None,,1020 m²,R 3739,,gauteng,johannesburg,rosebank-and-parktown,parkview
3,14 Bed House in Parktown North,https://www.privateproperty.co.za/for-sale/gau...,14,13,None,5,None,None,None,None,...,None,None,,2900 m²None,,,gauteng,johannesburg,rosebank-and-parktown,parktown-north
4,2 Bed Apartment in Gresswold,https://www.privateproperty.co.za/for-sale/gau...,2,2,2,None,None,✔,None,✔,...,None,None,72 m²RatesR 386,,R 386,,gauteng,johannesburg,rosebank-and-parktown,gresswold
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23764,6 Bed House in Windsor Glen,https://www.privateproperty.co.za/for-sale/gau...,6,5,None,1,None,None,None,None,...,None,None,,1554 m²,R 115,,gauteng,johannesburg,randburg-and-ferndale,windsor-glen
23765,2 Bed Apartment in Ferndale,https://www.privateproperty.co.za/for-sale/gau...,2,1,None,1,None,None,None,None,...,None,None,84 m²RatesR 445,,R 445,,gauteng,johannesburg,randburg-and-ferndale,ferndale
23766,2 Bed Apartment in Ferndale,https://www.privateproperty.co.za/for-sale/gau...,2,2,None,1,None,None,None,None,...,None,None,60 m²,56 m²,,,gauteng,johannesburg,randburg-and-ferndale,ferndale
23767,2 Bed Townhouse in Windsor West,https://www.privateproperty.co.za/for-sale/gau...,2,1,None,1,None,None,None,None,...,None,None,108 m²None,,,,gauteng,johannesburg,randburg-and-ferndale,windsor-west


In [22]:
final_list = final_list[final_list['Price'] != 'Sold']

# Create a folder named "Sold_Properties" if it doesn't exist
folder_name = 'Listed Properties'
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# Create the file name with today's date
file_name = f'Listed Properties/Current Listed Properties.csv'

# Save the DataFrame to CSV
sold_properties.to_csv(file_name, index=False)

# Create the file name with today's date
file_name = f'Listed Properties/Listed Properties {datetime.now().strftime("%Y-%m-%d")}.csv'

# Save the DataFrame to CSV
sold_properties.to_csv(file_name, index=False)

In [23]:
sold_properties = pd.read_csv('all_property_details.csv')

In [24]:
sold_properties = sold_properties[sold_properties['Price'] == 'Sold']
sold_properties['Sold Date'] = datetime.now().strftime('%Y-%m-%d')

# Get the list ID
# Function to extract list ID
def extract_list_id(url):
    return url.split('/')[-1]

# Apply the function to extract list IDs and store them in a new column
sold_properties['List ID'] = sold_properties['Href'].apply(lambda x: extract_list_id(x))

In [25]:
# Create a folder named "Sold_Properties" if it doesn't exist
folder_name = 'Sold Properties'
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# Create the file name with today's date
file_name = f'Sold Properties/Sold Properties {datetime.now().strftime("%Y-%m-%d")}.csv'

# Save the DataFrame to CSV
sold_properties.to_csv(file_name, index=False)